In [38]:
import numpy as np
from sklearn.metrics import log_loss,accuracy_score
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
import pandas as pd 
import xgboost as xgb
import json
from collections import Counter

import gc
from tqdm import tqdm
from itertools import combinations

from warnings import filterwarnings
filterwarnings('ignore')

In [39]:
train_df = pd.read_csv("train.csv",index_col='id')
test_df = pd.read_csv("test.csv")

train_df

,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Calories
id,,,,,,,,
0,male,36,189.0,82.0,26.0,101.0,41.0,150.0
1,female,64,163.0,60.0,8.0,85.0,39.7,34.0
2,female,51,161.0,64.0,7.0,84.0,39.8,29.0
3,male,20,192.0,90.0,25.0,105.0,40.7,140.0
4,female,38,166.0,61.0,25.0,102.0,40.6,146.0
...,...,...,...,...,...,...,...,...
749995,male,28,193.0,97.0,30.0,114.0,40.9,230.0
749996,female,64,165.0,63.0,18.0,92.0,40.5,96.0
749997,male,60,162.0,67.0,29.0,113.0,40.9,221.0


In [40]:
le = LabelEncoder()

categorical_cols = ['Sex']
all_cols = ['Sex', 'Age', 'Height', 'Weight', 'Duration', 'Heart_Rate', 'Body_Temp']

def feature_engineering(df):  
    
    for c in categorical_cols:
        #df.drop(c,axis=1,inplace=True)
        #df[c] = le.fit_transform(df[c]) # Converts categorical column into int format
        df[c] = df[c].astype('category') # Define column type as category """
        df[c] = df[c].cat.codes

    df['BMI'] = df['Weight'] / (df['Height'] / 100) ** 2
    df['Intensity'] = df['Heart_Rate'] / df['Duration']
    
    #df.drop(['Age','Height','Weight'],axis=1,inplace=True)
    #df["Body_Temp"] = np.expm1(df["Body_Temp"])
   
    #df["Body_Temp"] = df["Body_Temp"]

    df['Age_bin6'] = pd.qcut(df['Age'], q=6, labels=[1, 2, 3, 4, 5, 6]).astype(int)
    df['Height_log10p'] = np.log10(df['Height']).astype(float)
    df['Sex_Weight_prod'] = df['Sex'] * df['Weight']
    df['Sex_Age_log10p_sum'] = df['Sex'] + np.log10(df['Age']).astype(float)
    df['Sex_Age_bin6_sum'] = df['Sex'] + pd.qcut(df['Age'], q=6, labels=[1, 2, 3, 4, 5, 6]).astype(int)
    df['Sex_Duration_log10p_prod'] = df['Sex'] * np.log10(df['Duration']).astype(float)
    df['Sex_Body_Temp_log10p_prod'] = df['Sex'] * np.log10(df['Body_Temp']).astype(float)
    df['Age_Age_bin6_ratio'] = df['Age'] / pd.qcut(df['Age'], q=6, labels=[1, 2, 3, 4, 5, 6]).astype(int)
    df['Height_Height_log10p_prod'] = df['Height'] * np.log10(df['Height']).astype(float)
    df['Height_Height_bin6_ratio'] = df['Height'] / pd.qcut(df['Height'], q=6, labels=[1, 2, 3, 4, 5, 6]).astype(int)
    df['Height_Duration_log1p_sum'] = df['Height'] + np.log1p(df['Duration'].astype(float))
    df['Heart_Rate_log10p_Duration_ratio'] = np.log10(df['Heart_Rate']).astype(float) / df['Duration']
    df['Body_Temp_bin6_Duration_ratio'] = pd.qcut(df['Body_Temp'], q=6, labels=[1, 2, 3, 4, 5, 6]).astype(int) / df['Duration']
    df['Heart_Rate_Heart_Rate_log10p_prod'] = df['Heart_Rate'] * np.log10(df['Heart_Rate']).astype(float)
    df['Body_Temp_Height_log10p_prod'] = df['Body_Temp'] * np.log10(df['Height']).astype(float)
    df['Body_Temp_Height_log1p_ratio'] = df['Body_Temp'] / np.log1p(df['Height']).astype(float)
    df['Body_Temp_Heart_Rate_log10p_diff'] = df['Body_Temp'] - np.log10(df['Heart_Rate']).astype(float)
    df['Age_bin6_Weight_log10p_diff'] = pd.qcut(df['Age'], q=6, labels=[1, 2, 3, 4, 5, 6]).astype(int) - np.log10(df['Weight']).astype(float)

    gc.collect()
    
    """from sklearn.preprocessing import StandardScaler

    scaler = StandardScaler()
    df = pd.DataFrame(scaler.fit_transform(df[['Duration', 'Heart_Rate', 'Body_Temp']]),
                            columns=['Duration', 'Heart_Rate', 'Body_Temp'])"""
    
            
    return df


test_df = feature_engineering(test_df)
train_df = feature_engineering(train_df)

labels = train_df['Calories']
train_df.drop('Calories',axis=1,inplace=True)

train_df.describe()

KeyboardInterrupt: 

In [ ]:
#%pip install catboost
import catboost as cb
import numpy as np

# Original labels (before log transform)
original_labels = labels.copy()  # Preserve original labels
y = np.log1p(original_labels)    # Log-transform for training
X = train_df

# Custom RMSLE evaluation metric
class RMSLEMetric(object):
    def get_final_error(self, error, weight):
        return np.sqrt(error / (weight + 1e-38))

    def is_max_optimal(self):
        return False

    def evaluate(self, approxes, target, weight):
        # Convert log-scale predictions to original scale
        predictions = np.expm1(approxes[0])
        
        # Ensure original labels are in correct shape
        y_true = np.expm1(target)  # Reverse log1p transform
        
        # Calculate RMSLE
        log_diff = np.log1p(predictions) - np.log1p(y_true)
        squared_errors = np.square(log_diff)
        
        return np.sum(squared_errors), len(squared_errors)

params = {
    'verbose': 100,
    'task_type': 'CPU',
    'iterations': 5153,
    'learning_rate': 0.010855964248935647,
    'boosting_type': 'Plain',
    'depth': 10,
    'l2_leaf_reg': 7.3734014072244,
    'subsample': 0.25661056446892316,
    'colsample_bylevel': 0.6683485455329418,
    'min_data_in_leaf': 56,
    'random_seed': 1,
    'early_stopping_rounds': 100,
    'loss_function': 'RMSE',  # Keep training on RMSE of log-transformed values
    'eval_metric': RMSLEMetric(),  # But evaluate using RMSLE
    'cat_features': categorical_cols
}

# Create Pool with original labels reference
pool = cb.Pool(X, y, cat_features=categorical_cols)

# Cross-validation with RMSLE
cv_results = cb.cv(
    pool=pool,
    params=params,
    fold_count=5,
    partition_random_seed=1,
    shuffle=True,
    stratified=False,
    as_pandas=True,
    plot=True
)

# Best RMSLE score
best_rmsle = cv_results['test-RMSLE-mean'].min()
print(f"Best RMSLE: {best_rmsle:.4f}")

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Training on fold [0/5]
0:	learn: 4.2057749	test: 4.2058331	best: 4.2058331 (0)	total: 57.9ms	remaining: 4m 58s
100:	learn: 1.4242184	test: 1.4242582	best: 1.4242582 (100)	total: 6.41s	remaining: 5m 20s
200:	learn: 0.4888456	test: 0.4891104	best: 0.4891104 (200)	total: 12.4s	remaining: 5m 5s
300:	learn: 0.1800006	test: 0.1808294	best: 0.1808294 (300)	total: 18.4s	remaining: 4m 55s
400:	learn: 0.0892060	test: 0.0909095	best: 0.0909095 (400)	total: 24.4s	remaining: 4m 48s
500:	learn: 0.0687736	test: 0.0709961	best: 0.0709961 (500)	total: 30.4s	remaining: 4m 42s
600:	learn: 0.0641724	test: 0.0665587	best: 0.0665587 (600)	total: 36.5s	remaining: 4m 36s
700:	learn: 0.0624325	test: 0.0648741	best: 0.0648741 (700)	total: 42.6s	remaining: 4m 30s
800:	learn: 0.0614066	test: 0.0638864	best: 0.0638864 (800)	total: 48.7s	remaining: 4m 24s
900:	learn: 0.0606628	test: 0.0631983	best: 0.0631983 (900)	total: 55.2s	remaining: 4m 20s
1000:	learn: 0.0601183	test: 0.0626913	best: 0.0626913 (1000)	total: 1m

CatBoostError: catboost/python-package/catboost/helpers.cpp:57: Traceback (most recent call last):
  File "_catboost.pyx", line 1552, in _catboost._MetricEval
  File "/tmp/ipykernel_77263/3542763718.py", line 23, in evaluate
    y_true = np.expm1(target)  # Reverse log1p transform
             ^^^^^^^^^^^^^^^^
KeyboardInterrupt


In [ ]:
def prep_submission(ids,preds):

    submission_df = pd.DataFrame({
        'id': ids,
        'preds' : preds[:],

    })

    submission_df.to_csv("submission.csv", index=False)
    print("Submission file saved successfully!")